In [2]:
# Tuto inspiré largement de l'article publié sur Medium.com par Javed Shaikh
# Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK
#
# adapté en python 3.6
#
#https://medium.com/towards-data-science/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [3]:
### Import du dataset «The 20 newsgroups text»
# les textes sont stockées dans "data" et les attributs dans "target"
# plus d'infos sur la structure du dataset >> http://scikit-learn.org/stable/datasets/twenty_newsgroups.html

from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [4]:
# afficher la liste des catégories prédéfinies du classifier

print(twenty_train.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
# afficher les trois premieres lignes du premier texte 

print("\n".join(twenty_train.data[0].split("\n")[:3]))
# print(twenty_train.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [6]:
# afficher la categorie du premier texte

print(twenty_train.target_names[twenty_train.target[0]])

rec.autos


In [7]:
# dimension des tableaux

print(len(twenty_train.data))
print(len(twenty_train.target))

11314
11314


In [8]:
### extraction des features
# génération des bags of words
# matrice à 2 entrées (Document-Term matrix. [n_samples, n_features]) : 
# 1| les textes du dataset (samples), 2| les mots contenus dans le dataset (features)
# la valeur contenu est le nombre d'occurences du mot dans le texte

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

In [9]:
# calcul du TF-IDF : term frequency-inverse document frequency
# objectif : ponderer la fréquence du mot selon sa rareté dans le corpus de référence. 
# un mot fréquent dans un texte mais fréquent dans le corpus est peu signifiant. 
# un mot fréquent dans un texte mais rare dans le corpus est très signifiant. son poids est renforcé par la méthode TF-IDF

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [10]:
### execution d'algorithmes ML - Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# constitution du pipeline utilisé pour l'entrainement et pour le test
text_clf = Pipeline([('vect', CountVectorizer()), \
                     ('tfidf', TfidfTransformer()), \
                     ('clf', MultinomialNB()), \
                    ])
# entrainement
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

# test de la performance de la classification Naive Bayes
# import du dataset de test
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
# prediction avec le dataset de test
predicted = text_clf.predict(twenty_test.data)
# pourcentage de cas où la prediction est correcte
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [11]:
# determiner la performance optimale avec ‘GridSearchCV’ pour le Naive Bayes
# en combinant plusieurs valeurs pour chaque parametre de l'entrainement

from sklearn.model_selection import GridSearchCV

# liste de parametres pour le tuning
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], \
              'tfidf__use_idf': (True, False), \
              'clf__alpha': (1e-2, 1e-3), \
             }

# creation et execution de l'instance
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

In [12]:
# meilleur résultat de prediction (pourcentage)
gs_clf.best_score_

0.90675269577514583

In [13]:
# paramètres utilisés pour obtenir le meilleur résultat de prediction
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [16]:
gs_clf.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...near_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))])

In [20]:
# reuse des parametres obtenus
# cross validation prediction sur training set

from sklearn.model_selection import cross_val_predict
predictedCV = cross_val_predict(gs_clf.best_estimator_, twenty_train.data, twenty_train.target)
np.mean(predictedCV == twenty_train.target)

0.90675269577514583

In [21]:
# cross validation prediction sur test set

predictedCV = cross_val_predict(gs_clf.best_estimator_, twenty_test.data, twenty_test.target)
np.mean(predictedCV == twenty_test.target)

0.88542219861922467

In [24]:
# elimination des stop words / Naive Bayes

# from sklearn.pipeline import Pipeline
# enrichissement du pipeline NaiveBayes
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), \
                     ('tfidf', TfidfTransformer()), \
                     ('clf', MultinomialNB()), \
                    ])

# entrainement + prediction + évaluation
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.81691449814126393

In [25]:
# FitPrior=False, Whether to learn class prior probabilities or not. If false, a uniform prior will be used.

# from sklearn.pipeline import Pipeline
# enrichissement du pipeline NaiveBayes
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), \
                     ('tfidf', TfidfTransformer()), \
                     ('clf', MultinomialNB(fit_prior=False)), \
                    ])
# liste de parametres pour le tuning
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], \
              'tfidf__use_idf': (True, False), \
              'clf__alpha': (1e-2, 1e-3), \
             }

# entrainement + prediction + évaluation
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8214285714285714

In [55]:
### execution d'algorithmes ML - SVM (support vector machine)

from sklearn.linear_model import SGDClassifier

# constitution du pipeline utilisé pour l'entrainement et pour le test
# max_iter = 5 par défaut, l'entrainement pourrait être stoppé trop tôt : 82,38%
# test à 5000 iterations, mais sklearn n'utilise pas le GPU, donc qqes longueurs, faible amélio : 82,42%
# test à 100 itérations : 82.40%
text_clf_svm = Pipeline([('vect', CountVectorizer()), \
                         ('tfidf', TfidfTransformer()), \
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=100, random_state=42)), \
                         ])
# entrainement
_ = text_clf_svm.fit(twenty_train.data, twenty_train.target)
# prediction avec le dataset de test
predicted_svm = text_clf_svm.predict(twenty_test.data)
# pourcentage de cas où la prediction est correcte
np.mean(predicted_svm == twenty_test.target)

0.82408390865639936

In [56]:
# determiner la performance optimale avec ‘GridSearchCV’ pour le Support Vector Machine

# max_iter = 5 -> 89.79%
# max_iter = 100 -> 89.94%
#from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], \
                  'tfidf__use_idf': (True, False), \
                  'clf-svm__alpha': (1e-2, 1e-3), \
                 }
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_

0.8994166519356549

In [53]:
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [62]:
# elimination des stop words / SVM
# enrichissement du pipeline NaiveBayes
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range=(1, 2))), \
                         ('tfidf', TfidfTransformer()), \
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=100, random_state=42)), \
                         ])
# entrainement + prediction + évaluation
_ = text_clf_svm.fit(twenty_train.data, twenty_train.target)
predicted_svm = text_clf_svm.predict(twenty_test.data)
np.mean(predicted_svm == twenty_test.target)

0.83258098778544876